In [1]:
import normals_lib
import kaolin as kal
import torch
import numpy as np

#mesh = kal.rep.TriangleMesh.from_obj("model.obj") # Modelo de una esfera
mesh = kal.rep.TriangleMesh.from_obj("cow.obj") # Modelo de vaca
#mesh = kal.rep.TriangleMesh.from_obj("torus.obj")
normals = normals_lib.compute_simple_vertex_normals(mesh)
normals_lib.visualize_normals(mesh, normals)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [ ]:
import pointareas

pa, ca = pointareas.compute_pointareas(mesh)
pointareas.visualize_pointareas(mesh, pa)

f = open("/home/danniccs/rtsc-1.6/model_pointareas", "r")
tri_pa = [float(l) for l in f]
tri_pa_tens = torch.tensor(tri_pa)
pointareas.visualize_pointareas(mesh, tri_pa_tens)

In [2]:
# Calculate principal curvatures and curvature directions
import curvature

k1, k2, e1, e2 = curvature.compute_curvatures(mesh)

In [3]:
with open("/home/danniccs/rtsc-1.6/k1curv", "r") as f:
    tri_k1 = [float(l) for l in f]
tri_k1_tens = torch.tensor(tri_k1)

with open("/home/danniccs/rtsc-1.6/k2curv", "r") as f:
    tri_k2 = [float(l) for l in f]
tri_k2_tens = torch.tensor(tri_k2)

with open("/home/danniccs/rtsc-1.6/pdir1", "r") as f:
    splitLines = [l.replace('(', '').replace(')', '').split(', ') for l in f]
tri_e1 = [[float(el[0]), float(el[1]), float(el[2])] for el in splitLines]
tri_e1_tens = torch.tensor(tri_e1)

In [12]:
k1.mean()

tensor(2.5393)

In [13]:
tri_k1_tens.mean()

tensor(2.6917)

In [6]:
k2.mean()

tensor(0.2401)

In [7]:
tri_k2_tens.mean()

tensor(0.2150)

In [8]:
import meshplot as mp
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=k1.numpy(),
        shading={"colormap": "bwr"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [9]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=tri_k1_tens.numpy(),
        shading={"colormap": "bwr"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [10]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), k2.numpy(),
        shading={"colormap": "bwr"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [11]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), tri_k2_tens.numpy(),
        shading={"colormap": "bwr"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [17]:
normals_lib.visualize_normals(mesh, e1)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [18]:
normals_lib.visualize_normals(mesh, tri_e1_tens)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [24]:
print(torch.nn.functional.normalize(e1))
print('\n')
print(torch.nn.functional.normalize(tri_e1_tens))
print('\n')
print(e1.mean())
print('\n')
print(e2.mean())
print('\n')
dots = torch.zeros(e1.shape)
for i in range(0, e1.shape[0]):
    dots[i] = torch.dot(e1[i], tri_e1_tens[i])
print(dots.mean())

tensor([[-0.6558, -0.0081, -0.7549],
        [ 0.0457,  0.8684, -0.4937],
        [ 0.5275, -0.7094,  0.4673],
        ...,
        [-0.2513, -0.8557, -0.4523],
        [ 0.0842, -0.4508, -0.8886],
        [-0.3399, -0.8706,  0.3558]])


tensor([[-0.6676, -0.0402, -0.7434],
        [-0.7703, -0.2803, -0.5727],
        [ 0.5335, -0.7041,  0.4687],
        ...,
        [-0.2589, -0.8611, -0.4376],
        [ 0.0854, -0.4481, -0.8899],
        [-0.3350, -0.8911,  0.3060]])


tensor(0.0013)


tensor(-0.0029)


tensor(0.9377)
